<a href="https://colab.research.google.com/github/daveyeajoon/sales-prediction/blob/main/Sales_Prediction_Part5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.compose import make_column_selector, make_column_transformer
set_config(display='diagram')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sales_predictions.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [13]:
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088


In [14]:
df.dropna().shape[0] / df.shape[0]

1.0

Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.  


In [17]:
y = df['Item_Outlet_Sales']

X = df.drop(columns = ['Item_Outlet_Sales', 'Item_Identifier'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
df['Item_Outlet_Sales'].value_counts()

1342.2528    11
1438.1280    11
958.7520     10
703.0848     10
1845.5976     9
             ..
256.3330      1
3941.5360     1
3858.9768     1
5496.8448     1
2778.3834     1
Name: Item_Outlet_Sales, Length: 2449, dtype: int64

Create a preprocessing object to prepare the dataset for Machine Learning

In [19]:
categoricals = X_train.select_dtypes(include = 'object')
for col in categoricals.columns:
  print(col)
  print(categoricals[col].value_counts(), '\n')

Item_Fat_Content
Low Fat    2101
Regular    1161
LF          132
reg          54
low fat      39
Name: Item_Fat_Content, dtype: int64 

Item_Type
Snack Foods              525
Fruits and Vegetables    495
Household                378
Frozen Foods             343
Dairy                    281
Canned                   277
Baking Goods             259
Health and Hygiene       217
Soft Drinks              179
Meat                     163
Hard Drinks               87
Breads                    85
Others                    72
Starchy Foods             63
Breakfast                 39
Seafood                   24
Name: Item_Type, dtype: int64 

Outlet_Identifier
OUT018    707
OUT013    701
OUT049    697
OUT046    691
OUT035    691
Name: Outlet_Identifier, dtype: int64 

Outlet_Size
Medium    1404
Small     1382
High       701
Name: Outlet_Size, dtype: int64 

Outlet_Location_Type
Tier 3    1408
Tier 1    1388
Tier 2     691
Name: Outlet_Location_Type, dtype: int64 

Outlet_Type
Supermarket Type1 

In [20]:
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')

scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [21]:
num_pipe = make_pipeline(mean_imputer,scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [22]:
cat_pipe = make_pipeline(freq_imputer, ohe)
cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [25]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)



In [26]:
preprocessor = make_column_transformer(num_tuple, cat_tuple, remainder='drop')

In [27]:
preprocessor.fit(X_train)


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fecdff66990>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fecdff66a90>)])

In [28]:
x_train_process = preprocessor.transform(X_train)
x_train_process

array([[-1.61873556, -0.80142192,  1.55615978, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.6567808 ,  1.06509895,  0.31746464, ...,  0.        ,
         1.        ,  0.        ],
       [-0.49444827,  1.49877652, -0.31978717, ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [ 0.28752511,  0.43911456, -0.98967997, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.21264393, -0.69794226,  1.19135799, ...,  0.        ,
         1.        ,  0.        ],
       [-0.08688075, -0.28145496, -1.30541276, ...,  1.        ,
         1.        ,  0.        ]])

In [29]:
x_test_process = preprocessor.transform(X_test)
x_test_process

array([[-1.01540727, -0.60128412, -0.45554685, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.49632115,  0.96133869, -0.58425587, ...,  0.        ,
         1.        ,  0.        ],
       [-0.66560523, -0.73027623, -0.39672503, ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [-1.4925073 , -0.86456805,  0.55566907, ...,  0.        ,
         1.        ,  0.        ],
       [-0.81536757,  1.2618797 ,  0.28537243, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.60329425,  1.80207477, -0.83529718, ...,  0.        ,
         1.        ,  0.        ]])